# Dash Interface

## Installing required libraries

In [ ]:
pip install jupyter-dash -q

     |████████████████████████████████| 1.1 MB 5.8 MB/s 
     |████████████████████████████████| 3.7 MB 14.0 MB/s 
     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 1.8 MB 34.7 MB/s 
     |████████████████████████████████| 357 kB 47.7 MB/s 


In [ ]:
pip install dash-cytoscape -q

     |████████████████████████████████| 3.6 MB 4.2 MB/s 


In [ ]:
pip install plotly --upgrade

     |████████████████████████████████| 20.6 MB 1.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
pip install -U dash-canvas

     |████████████████████████████████| 737 kB 5.4 MB/s 
  Created wheel for dash-canvas: filename=dash_canvas-0.1.0-py3-none-any.whl size=745346 sha256=4e7e3da5628caac66dfa52506c3d2a5daf2c59797b84b550751c41e47253432e
  Stored in directory: /root/.cache/pip/wheels/82/4d/c5/f0cec1a0ce8d7930976eccca658a39c9e3a3c62867ff575f09
Successfully built dash-canvas


## Importing modules 

In [ ]:
from os.path import dirname, join, isfile
from itertools import chain
import pandas as pd
import numpy as np
import scipy.signal
import scipy.io as sio
import seaborn as sns
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash  # pip install dash
import dash_cytoscape as cyto  # pip install dash-cytoscape==0.2.0 or higher
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input
import pandas as pd  # pip install pandas
import plotly.express as px
import math
from dash import no_update
from dash_canvas import DashCanvas
import base64
sns.set_theme(style="darkgrid")
%matplotlib inline


## Function to get dataframe

In [ ]:
def get_dataframe(mat_fpath):
    ''' Obtain Pandas dataframe from mat file
    :param mat_fpath: File path to the mat file from which pandas DF should be created
    '''
    assert isfile(mat_fpath), "Invalid file path!"
    mat_contents = sio.loadmat(mat_fpath)
    viz_data = mat_contents["ccFddData"]
    viz_data = mat_contents['ccFddData']
    n = 5;
    iFDD_names = ["Sys2_Impact_Location_P{}".format(str(i+1)) for i in range(n)]
    dFDD_pnames = ["Sys2_Dmg_P_{}".format(str(i+1)) for i in range(n)]
    dFDD_cpnames = ["Sys2_Dmg_P_{}_c".format(str(i+1)) for i in range(n)]
    dFDD_names = list(chain.from_iterable(zip(dFDD_pnames, dFDD_cpnames)))
    col_names = ['Time', 'Sys2_Total_Dmg', 'Sys2_SimFDD_xe'] + iFDD_names + dFDD_names
    df = pd.DataFrame(viz_data, columns = col_names)
    return df             

In [ ]:
file_name = 'newData_v6.mat'
mat_fpath = join(dirname('./'), 'data', file_name)
data = get_dataframe("/content/newData_v6.mat")

In [ ]:
t = data['Time'].to_numpy()
trueHS = data['Sys2_Total_Dmg'].to_numpy()
sim_fdd_xe = data['Sys2_SimFDD_xe'].to_numpy()
impact_fdd = np.vstack(([data['Sys2_Impact_Location_P{}'.format(ii+1)].to_numpy() for ii in range(5)])).T
dmg_fdd = [np.vstack((data['Sys2_Dmg_P_{}'.format(ii+1)].to_numpy(), \
                     data['Sys2_Dmg_P_{}_c'.format(ii+1)].to_numpy())).T for ii in range(5)]

## Calculating time of impact and the probability around it

In [ ]:
x = (int(np.argmax(impact_fdd))) / 5 #Getting the index for the peak registered for the impact FDD
timeofImpact = int(x)
timeofImpact

prob = (impact_fdd[timeofImpact]) #list with impact probabilities for all 5 segments
domeaff = [] #list with index of segments in which impact was registered
for i in range(5):
  if prob[i] > 0.2:
    domeaff.append(i)

#translating prabiblities into words
for i in domeaff:
  if prob[i] < 0.4:
    confidence = "probably not"
  elif prob[i] < 0.6:
    confidence = "chances about even"
  elif prob[i] < 0.8:
    confidence = "probablt"
  else:
    confidence = "almost certainly" 



## Creating lists to store information

In [ ]:
height = np.zeros(5) #list that will store the average values
stdv = np.zeros(5) #list that will store standard deviation values
min_ = np.zeros(5) #list that will contain the min values for the data 
max_ = np.zeros(5) #list that will contain the max values for the data
st_error = np.zeros(5) #list that contains the standard error

## Function to calculate statiscs of data

In [ ]:
def avg_std2(dmg_info, impact, num, sec1, sec2): #function that calculates the mean between to time intervals
  a = (dmg_info)
  b = a[(sec1 * 1000):(sec2 * 1000),0]
  min_[num] = np.min(b) #appends minimum value for each segment
  max_[num] = np.max(b) #appends maximum value for each segment
  height[num] = np.mean(b) #calculates mean of set of values
  stdv[num] = np.std(b) #calculates std dev. of set of values
  st_error = np.std(b, ddof=1) / np.sqrt(np.size(b)) #calculates the standard error for each segment
  num += 1;
  return num;

## Function to cap std on bar chart graph at 0 and 1

In [ ]:
def lin(std, h): #function that limit standard deviations to between one and zero
# takes standard deviation and height of the bars and makes sure the height +- the standard
#deviation is within that range. Returns list with values to use
  fr = np.zeros(10)
  index = 0
  for i in std:
    if index < 5:
      if (h[index] - i) < 0:
        fr[index] = h[index]
      else:
        fr[index] = i
    else:
      if (h[index] + i > 1):
        fr[index] = 1 - h[index]
      else:
        fr[index] = i
    index = index + 1

  return (fr)

## Function to call other functions

In [ ]:
def pregraph2(t1, t2):
#function that calls avg_std2 function, and adjusts the standard deviation
#to be within 0 and 1

  count = 0;
  counter = avg_std2(dmg_fdd[0], timeofImpact, count, t1, t2);
  counter = avg_std2(dmg_fdd[1], timeofImpact, counter, t1, t2);
  counter = avg_std2(dmg_fdd[2], timeofImpact, counter, t1, t2);
  counter = avg_std2(dmg_fdd[3], timeofImpact, counter, t1, t2);
  counter = avg_std2(dmg_fdd[4], timeofImpact, counter, t1, t2);
  aaa = np.concatenate((stdv, stdv))
  bbb = np.concatenate((height, height))
  s = lin(aaa, bbb)
  c = np.array_split(s, 2)
  return c

## Function to graph bar charts

In [ ]:
def graph3(start_time, end_time): #function that graphs results, returns figure
  ccc = pregraph2(start_time, end_time)
  print(len(ccc[0]))
  seg = ["segment 1", "segment 2", "segment 3", "segment 4", "segment 5"]
  fig2 = px.bar(x=seg, y=height, error_y=ccc[1], error_y_minus=ccc[0],
                title=f"Probability of damage in each segment from {start_time} seconds until {end_time} seconds",
                range_y = [0, 1], labels={"y": "P [Damage]", "x": ""})
            
  
  
  return fig2

## Function to graph FDD data

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
#statically graphs FDD data in a subplot
def FDD_graphs():
  fig = make_subplots(rows=3, cols=2, 
                      subplot_titles=("Damage FDD: Segment 1", "Damage FDD: Segment 2", 
                      "Damage FDD: Segment 3", "Damage FDD: Segment 4", 
                      "Damage FDD: Segment 5"))


  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[0][:,0], name="P[Damage] seg 1"),
      row = 1, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[0][:,1], name="P[No Damage] seg 1"),
      row = 1, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[1][:,0], name="P[Damage] seg 2"),
      row = 1, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[1][:,1], name="P[No Damage] seg 2"),
      row = 1, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[2][:,0], name="P[Damage] seg 3"),
      row = 2, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[2][:,1], name="P[No Damage] seg 3"),
      row = 2, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[3][:,0], name="P[Damage] seg 4"),
      row = 2, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[3][:,1], name="P[No Damage] seg 4"),
      row = 2, col = 2
   )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[4][:,0], name="P[Damage] seg 5"),
      row = 3, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[4][:,1], name="P[No Damage] seg 5"),
      row = 3, col =1
  )


  #fig.update_layout(height=1000, width=900, title_text="FDD data for damage in each segment")
  return fig

## Creating explanation figure for bar and line chart
Gets pngs from github repository in which pictures are uploaded

In [ ]:
# Create figure
fig34 = go.Figure()

# Constants
img_width = 2339
img_height = 1654
scale_factor = 0.5

# Add invisible scatter trace.
# This trace is added to help the autoresize logic work.
fig34.add_trace(
    go.Scatter(
        x=[0, img_width * scale_factor],
        y=[0, img_height * scale_factor],
        mode="markers",
        marker_opacity=0
    )
)

# Configure axes
fig34.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor]
)

fig34.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x"
)

# Add image
fig34.add_layout_image(
    dict(
        x=0,
        sizex=img_width * scale_factor,
        y=img_height * scale_factor,
        sizey=img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source="https://raw.githubusercontent.com/BernardoAlvarenga/picture/main/test%20(1)-1.png")
)

# Configure other layout
fig34.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

# Disable the autosize on double click because it adds unwanted margins around the image
# More detail: https://plotly.com/python/configuration-options/
fig34.show(config={'doubleClick': 'reset'})

In [ ]:
# Create figure
fig35 = go.Figure()

# Constants
img_width = 2339
img_height = 1654
scale_factor = 0.5

# Add invisible scatter trace.
# This trace is added to help the autoresize logic work.
fig35.add_trace(
    go.Scatter(
        x=[0, img_width * scale_factor],
        y=[0, img_height * scale_factor],
        mode="markers",
        marker_opacity=0
    )
)

# Configure axes
fig35.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor]
)

fig35.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x"
)

# Add image
fig35.add_layout_image(
    dict(
        x=0,
        sizex=img_width * scale_factor,
        y=img_height * scale_factor,
        sizey=img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source="https://raw.githubusercontent.com/BernardoAlvarenga/picture/main/Line%20Graphs%20Explanation-1.png")
)

# Configure other layout
fig35.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

# Disable the autosize on double click because it adds unwanted margins around the image
# More detail: https://plotly.com/python/configuration-options/
fig35.show(config={'doubleClick': 'reset'})

## Creating Line Graphs (real time + static)

In [ ]:
from itertools import count
#Initializations of arrays and values for calculations
x = (int(np.argmax(impact_fdd))) / 5 #Getting the index for the peak registered for the impact FDD
timeofImpact = int(x)
default_interval = 3.0*1000 #Default interval of time for averaging and displaying on line graph
sec_ai = 60 #seconds after impact calculations will be done


cnt = 0

def pregraph4(it, ft):
  global average
  average = [[], [], [], [], []]
  global stddev
  stddev = [[], [], [], [], []]
  cnt = 0
  global low_1
  global low_2
  global low_3
  global low_4
  global low_5
  global high_1
  global high_2
  global high_3
  global high_4
  global high_5
  low_1 = []
  low_2 = []
  low_3 = []
  low_4 = []
  low_5 = []
  high_1 = []
  high_2 = []
  high_3 = []
  high_4 = []
  high_5 = []
  

  #Calculations of averages of windows over time, values stored in average and stddev lists
  def avg_std(dmg_info, impact, cnt, st, ft): #function that calculates the mean and standard deviation
  # for a certain time interval, can be adjusted
    a = (dmg_info[st*1000:])
    b = a[:((ft-st) * 1000),0]
    
    for i in range(0, len(b), 1000):
        
        if (i - default_interval) < 0:
            #print(np.mean(b[impact:i]))
            if (i == 0):
              average[cnt].append(np.mean(b[0]))
              stddev[cnt].append(np.std(b[0]))
            else:
              average[cnt].append(np.mean(b[0:i]))
              stddev[cnt].append(np.std(b[0:i]))
        else:
            average[cnt].append(np.mean(b[(i-(int(3.0*1000))):i]))
            stddev[cnt].append(np.std(b[(i-(int(3.0*1000))):i]))

  #Function Calls and List Initializations
  avg_std(dmg_fdd[0], timeofImpact, 0, it, ft);
  avg_std(dmg_fdd[1], timeofImpact, 1, it, ft);
  avg_std(dmg_fdd[2], timeofImpact, 2, it, ft);
  avg_std(dmg_fdd[3], timeofImpact, 3, it, ft);
  avg_std(dmg_fdd[4], timeofImpact, 4, it, ft);


  #Appending lower and upper bounds to repsective lists, capping at 0 and 1
  for i in range(len(average[0])):
    low_1.append(average[0][i] - stddev[0][i])
    low_2.append(average[1][i] - stddev[1][i])
    low_3.append(average[2][i] - stddev[2][i])
    low_4.append(average[3][i] - stddev[3][i])
    low_5.append(average[4][i] - stddev[4][i])
    high_1.append(average[0][i] + stddev[0][i])
    high_2.append(average[1][i] + stddev[1][i])
    high_3.append(average[2][i] + stddev[2][i])
    high_4.append(average[3][i] + stddev[3][i])
    high_5.append(average[4][i] + stddev[4][i])
    low_list = [low_1, low_2, low_3, low_4, low_5]
    high_list = [high_1, high_2, high_3, high_4, high_5]
    for k in low_list:
      if k[i] < 0:
        k[i] = 0
    for j in high_list:
      if j[i] > 1:
        j[i] = 1

  #Low lists reversed
  low_1 = low_1[::-1]
  low_2 = low_2[::-1]
  low_3 = low_3[::-1]
  low_4 = low_4[::-1]
  low_5 = low_5[::-1]

  #Iteration for x axis list
  index = count(start=it)
  global xvals
  xvals = [0]*len(average[0]) 
  for i in range(len(average[0])):
    xvals[i] = next(index)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def avg_linegraphs(it, ft):
    pregraph4(it, ft)
    fig = go.Figure()   
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_1+low_1,
        fill='toself',
        fillcolor='rgba(34, 139, 34, 0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 1',
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_2+low_2,
        fill='toself',
        fillcolor='rgba(0,176,246,0.2)',
        line_color='rgba(255,255,255,0)',
        name='Segment 2',
        showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_3+low_3,
        fill='toself',
        fillcolor='rgba(255,211,67,0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 3',
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_4+low_4,
        fill='toself',
        fillcolor='rgba(214, 39, 40, 0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 4',
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_5+low_5,
        fill='toself',
        fillcolor='rgba(255, 127, 14, 0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 5',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[0],
        line_color='rgb(34, 139, 34)',
        name='Segment 1',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[1],
        line_color='rgb(0,176,246)',
        name='Segment 2',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[2],
        line_color='rgb(255,211,67)',
        name='Segment 3',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[3],
        line_color='rgb(214, 39, 40)',
        name='Segment 4',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[4],
        line_color='rgb(255, 127, 14)',
        name='Segment 5',
    ))
    

    fig.update_traces(mode='lines')
    return fig

In [ ]:
#This function is called when the "real time data" tab is chosen. It arranges the datasets of averages and standard deviations needed when displaying the updating line graphs.
def avg_linegraphs_real_time():
    from itertools import count
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    #Initializations of arrays and values for calculations
    x = (int(np.argmax(impact_fdd))) / 5 #Getting the index for the peak registered for the impact FDD
    timeofImpact = int(x)
    default_interval = 3.0*1000 #Default interval of time for averaging and displaying on line graph
    sec_ai = 60 #seconds after impact calculations will be done
    average = [[], [], [], [], []]
    stddev = [[], [], [], [], []]
    cnt = 0

    #Calculations of averages of windows over time, values stored in average and stddev lists
    #Organization of datasets and calculations of means and std deviations across timeset
    def avg_std(dmg_info, impact, cnt): #function that calculates the mean and standard deviation
    # for a certain time interval, can be adjusted
      a = (dmg_info[0:])
      b = a[:(120 * 1000),0]
      for i in range(0, len(b), 1000):
          if (i - default_interval) < 0:
              #print(np.mean(b[impact:i]))
              if (i == 0):
                average[cnt].append(np.mean(b[0]))
                stddev[cnt].append(np.std(b[0]))
              else:
                average[cnt].append(np.mean(b[0:i]))
                stddev[cnt].append(np.std(b[0:i]))
          else:
              average[cnt].append(np.mean(b[(i-(int(3.0*1000))):i]))
              stddev[cnt].append(np.std(b[(i-(int(3.0*1000))):i]))

    #Function Calls and List Initializations
    #Initial lists are filled with average and deviation values for all 5 segments, hence 5 function calls
    avg_std(dmg_fdd[0], timeofImpact, 0);
    avg_std(dmg_fdd[1], timeofImpact, 1);
    avg_std(dmg_fdd[2], timeofImpact, 2);
    avg_std(dmg_fdd[3], timeofImpact, 3);
    avg_std(dmg_fdd[4], timeofImpact, 4);
    low_1 = []
    low_2 = []
    low_3 = []
    low_4 = []
    low_5 = []
    high_1 = []
    high_2 = []
    high_3 = []
    high_4 = []
    high_5 = []

    #Appending lower and upper bounds to repsective lists, capping at 0 and 1
    #Lists used later for error bands on line graph
    for i in range(len(average[0])):
      low_1.append(average[0][i] - stddev[0][i])
      low_2.append(average[1][i] - stddev[1][i])
      low_3.append(average[2][i] - stddev[2][i])
      low_4.append(average[3][i] - stddev[3][i])
      low_5.append(average[4][i] - stddev[4][i])
      high_1.append(average[0][i] + stddev[0][i])
      high_2.append(average[1][i] + stddev[1][i])
      high_3.append(average[2][i] + stddev[2][i])
      high_4.append(average[3][i] + stddev[3][i])
      high_5.append(average[4][i] + stddev[4][i])
      low_list = [low_1, low_2, low_3, low_4, low_5]
      high_list = [high_1, high_2, high_3, high_4, high_5]
      for k in low_list:
        if k[i] < 0:
          k[i] = 0
      for j in high_list:
        if j[i] > 1:
          j[i] = 1

    #Iteration for x axis list
    index = count()
    xvals = [0]*len(average[0])
    for i in range(len(average[0])):
      xvals[i] = next(index)

    #Loading data into frames
    #Multiple traces containing data for probability averages of 5 segments are plotted over time, along with 2 higher and lower error bands displaying standard deviation as a 
    #mathematical metric/visualization of uncertainty. Area between upper and lower error bands are shaded
    frames = []
    for frame in range(1, 121):
      #Each frame contains an extra second of data and is appended to frames list
      curr_frame = go.Frame(data = [go.Scatter(x = xvals[0:frame], y = high_1[0:frame], line = dict(color='rgba(34, 139, 34, 0.01)', width=4, dash='dash'), showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = low_1[0:frame], line = dict(color='rgba(34, 139, 34, 0.01)', width=4, dash='dash'), fillcolor='rgba(34, 139, 34, 0.2)', fill='tonexty', showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = high_2[0:frame], line = dict(color='rgba(0, 176, 246, 0.01)', width=4, dash='dash'), showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = low_2[0:frame], line = dict(color='rgba(0, 176, 246, 0.01)', width=4, dash='dash'), fillcolor='rgba(0, 176, 246, 0.2)', fill='tonexty', showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = high_3[0:frame], line = dict(color='rgba(255, 211, 67, 0.01)', width=4, dash='dash'), showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = low_3[0:frame], line = dict(color='rgba(255, 211, 67, 0.01)', width=4, dash='dash'), fillcolor='rgba(255, 211, 67, 0.2)', fill='tonexty', showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = high_4[0:frame], line = dict(color='rgba(214, 39, 40, 0.01)', width=4, dash='dash'), showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = low_4[0:frame], line = dict(color='rgba(214, 39, 40, 0.01)', width=4, dash='dash'), fillcolor='rgba(214, 39, 40, 0.2)', fill='tonexty', showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = high_5[0:frame], line = dict(color='rgba(255, 127, 14, 0.01)', width=4, dash='dash'), showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = low_5[0:frame], line = dict(color='rgba(255, 127, 14, 0.01)', width=4, dash='dash'), fillcolor='rgba(255, 127, 14, 0.2)', fill='tonexty', showlegend=False),
                                    go.Scatter(x = xvals[0:frame], y = (average[0])[0:frame], line_color='rgb(34, 139, 34)', name='Segment 1', mode = 'lines'),
                                    go.Scatter(x = xvals[0:frame], y = (average[1])[0:frame], line_color='rgb(0,176,246)', name='Segment 2', mode = 'lines'),
                                    go.Scatter(x = xvals[0:frame], y=(average[2])[0:frame], line_color='rgb(255,211,67)', name='Segment 3', mode = 'lines'),
                                    go.Scatter(x = xvals[0:frame], y=(average[3])[0:frame], line_color='rgb(214, 39, 40)', name='Segment 4', mode = 'lines'),
                                    go.Scatter(x = xvals[0:frame], y=(average[4])[0:frame], line_color='rgb(255, 127, 14)', name='Segment 5', mode = 'lines')])
      frames.append(curr_frame)
    
    #Figure is animated and starts with play button 
    fig = go.Figure(data = [go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines'),
                            go.Scatter(x = np.array([1]), y = np.array([0]), mode = 'lines')
                            ],
                    layout = {'title':'Average Probabilities of Damage Over Time',
                              'updatemenus':[{'type':'buttons',
                                              'buttons':[{
                                                  'label':'Play',
                                                  'method':'animate',
                                                  'args':[None]}]}]
                              },
                    frames = frames
                    )
    return fig
    

In [ ]:
avg_linegraphs_real_time()

## Creating real time impact FDD graphs
For this current iteration impact fdd data had to be simplified to accomodate for animation. Only 1 point for each second is shown, even though there are 1000 data points per second

In [ ]:
seg1d = []
for i in range(120000):
  if (i % 1000) == 0:
     num = np.average(impact_fdd[i:(i + 1000),0])
     seg1d.append(num)
seg1d[60] = 1
seg2d = []
for i in range(120000):
  if (i % 1000) == 0:
     num = np.average(impact_fdd[i:(i + 1000),1])
     seg2d.append(num)
seg3d = []
for i in range(120000):
  if (i % 1000) == 0:
     num = np.average(impact_fdd[i:(i + 1000),2])
     seg3d.append(num)
seg4d = []
for i in range(120000):
  if (i % 1000) == 0:
     num = np.average(impact_fdd[i:(i + 1000),3])
     seg4d.append(num)
seg5d = []
for i in range(120000):
  if (i % 1000) == 0:
     num = np.average(impact_fdd[i:(i + 1000),4])
     seg5d.append(num)

In [ ]:
fig43 = make_subplots(
    rows=5, cols=1, subplot_titles=('Segment 1', 'Segment 2', 'Segment 3', 'Segment 4', 'Segment 5'),
    horizontal_spacing=0.051, shared_xaxes = True, shared_yaxes = True
)

fig43.add_trace(go.Scatter(x=[0], y = [0], mode="lines", name="P[Damage] seg1"), row=1, col=1) #this is the trace of index 0

fig43.add_trace(go.Scatter(x=[0], y = [0], mode="lines", name="P[Damage] seg2"), row=2, col=1)  #trace of index 1
fig43.add_trace(go.Scatter(x=[0], y = [0], mode="lines", name="P[Damage] seg3"), row=3, col=1) #this is the trace of index 2

fig43.add_trace(go.Scatter(x=[0], y = [0], mode="lines", name="P[Damage] seg4"), row=4, col=1)  #trace of index 3

fig43.add_trace(go.Scatter(x=[0], y = [0], mode="lines", name="P[Damage] seg5"), row=5, col=1) #this is the trace of index 4


#traces=[0, 1, 2]` in the frame definition makes the difference: it tells that 
#the traces of index 0, 1 from the subplot(1,1), are unchanged, and we only ensure their visibility in each #frame (because neither x nor y are modified)
#while the trace 2 from the subplot(1,2) is animated, because the y-values are changed. 

frames_test =[go.Frame(data=[go.Scatter(x = np.arange(k), y = seg1d[0:k], mode = "lines"),
                        go.Scatter(x = np.arange(k), y = seg2d[0:k], mode = "lines"),
                        go.Scatter(x = np.arange(k), y = seg3d[0:k], mode = "lines"),
                        go.Scatter(x = np.arange(k), y = seg4d[0:k], mode = "lines"),
                         go.Scatter(x = np.arange(k), y = seg5d[0:k], mode = "lines")],
                  traces=[0,1, 2, 3, 4]) for k in range(120)]

fig43.frames=frames_test
button = dict(
             label='Play',
             method='animate',
             args=[None, dict(frame=dict(duration=50, redraw=False), 
                              transition=dict(duration=0),
                              fromcurrent=True,
                              mode='immediate')])
fig43.update_layout(updatemenus=[dict(type='buttons',
                              showactive=False,
                              y=0,
                              x=1.05,
                              xanchor='left',
                              yanchor='bottom',
                              buttons=[button] )
                                      ],
                 width=800, height=500)
                              
fig43.update_layout(xaxis_range=[0,120],yaxis2_range=[0,1], yaxis_range=[0,1],
                    yaxis3_range=[0,1], yaxis5_range=[0,1],yaxis4_range=[0,1],
                    title="Impact probabilities for each segment", xaxis5_title="Time",
                    yaxis3_title="P[Damage]")
fig43.show()  

## DASH interface 1.0

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([   
  html.Div(id='initial text', children=f"An impact {confidence} has occured at {timeofImpact/1000} seconds"),
  dcc.Dropdown(
      id='my_dropdown',
      options=[
                {"label": "See real time feedback", "value": "Sys2_Total_Dmg"},
                {"label": "Show estimated damage", "value": "Sys2_SimFDD_xe"},   
                {"label": "See real time FDD data", "value": "Sys2_Impact_Location_P2"}, 
                ],
      optionHeight=35,                    #height/space between dropdown options
      value='Sys2_SimFDD_xe',                    #dropdown value selected automatically when page loads
      disabled=False,                     #disable dropdown value selection
      multi=False,                        #allow multiple dropdown values to be selected
      searchable=True,                    #allow user-searching of dropdown values
      search_value='',                    #remembers the value searched in dropdown
      placeholder='Please select...',     #gray, default text shown when no option is selected
      clearable=True,                     #allow user to removes the selected value
      style={'width':"100%", 'display': 'inline-block'},             #use dictionary to define CSS styles of your dropdown         
    ),
    dcc.Graph(
        id="graph_selected",
        figure={}
    ),
    
    dcc.RangeSlider(
        id="time_interval",
        min=0,
        max=120,
        step=1,
        value=[0, 120],
        marks={
            0: "0",
            120: "120"
        }
        
    ),
    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='How estimated damage was calculated', value='tab-1' ,children=
                dcc.Graph(
                    id="exp1",
                    figure=fig34
                    )
                ),
        dcc.Tab(label='How FDD data was calculated', value='tab-2'),
        dcc.Tab(label="How real time feedback was calculated", value="tab-3", children=
                dcc.Graph(
                    id="exp2",
                    figure = fig35
                )
                )
         ]),
    html.Div(id='dd-output-container')
  ])

@app.callback(
    [Output('dd-output-container', 'children'), Output("graph_selected", "figure")],
    [Input('my_dropdown', 'value'), Input("time_interval", "value"), Input("tabs-example", "value")]
    
    )
     
def update_output(value, time_int, tabss):
    if value == "Sys2_SimFDD_xe":
      fig = graph3(time_int[0], time_int[1])
    elif value == "Sys2_Impact_Location_P2":
      fig = FDD_graphs()
    elif value == "Sys2_Total_Dmg":
      fig = avg_linegraphs()
    if tabss == "tab-1":
      tab_tr = html.Div([html.H3('Tab content 1')])
    elif tabss == "tab-2":
      tab_tr = html.Div([html.H3('Tab content 2')])
    

    return tab_tr, fig



app.run_server(debug=False, port=8040)


 * Running on http://127.0.0.1:8040/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2021 15:25:51] "GET /_alive_582c4451-2171-47a2-beb5-5d5c7dcf6eb8 HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>

## Dash interface 2.0

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
  dcc.Tabs(id='tabs-main', value='tab-dmg', children=[
        dcc.Tab(label='After impact Info', value='tab-main' ,children=
                html.Div([html.Div(id='initial text', children=f"An impact {confidence} has occured at {timeofImpact/1000} seconds. Showing last 2 minutes of data"),
                dcc.Dropdown(
                    id='my_dropdown',
                    options=[
                              {"label": "See damage from past 2 minutes", "value": "Sys2_Total_Dmg"},
                              {"label": "Show estimated damage", "value": "Sys2_SimFDD_xe"},   
                              {"label": "See FDD data for the past 2 minutes", "value": "Sys2_Impact_Location_P2"}, 
                              ],
                    optionHeight=35,                    #height/space between dropdown options
                    value='Sys2_SimFDD_xe',                    #dropdown value selected automatically when page loads
                    disabled=False,                     #disable dropdown value selection
                    multi=False,                        #allow multiple dropdown values to be selected
                    searchable=True,                    #allow user-searching of dropdown values
                    search_value='',                    #remembers the value searched in dropdown
                    placeholder='Please select...',     #gray, default text shown when no option is selected
                    clearable=True,                     #allow user to removes the selected value
                    style={'width':"100%", 'display': 'inline-block'},             #use dictionary to define CSS styles of your dropdown         
                  ),
                  dcc.Graph(
                      id="graph_selected",
                      figure={}
                  ),
                  
                  dcc.RangeSlider(
                      id="time_interval",
                      min=0,
                      max=120,
                      step=1,
                      value=[0, 120],
                      marks={
                          0: "-120",
                          120: "0"
                      }
                      
                  ),
                  dcc.Tabs(id='tabs-example', value='tab-1', children=[
                      dcc.Tab(label='How estimated damage was calculated', value='tab-1' ,children=
                              dcc.Graph(
                                  id="exp1",
                                  figure=fig34
                                  )
                              ),
                      dcc.Tab(label='How damage FDD data was calculated', value='tab-2'),
                      dcc.Tab(label="How real time feedback was calculated", value="tab-3", children=
                              dcc.Graph(
                                  id="exp2",
                                  figure = fig35
                              )
                              ),
                      dcc.Tab(label="How Impact FDD data was calculated", value="tab4")
                      ]),
                  html.Div(id='dd-output-container')])
                              ),
        dcc.Tab(label="Real time status", value="tab-dmg", children=
                html.Div([
                          dcc.Graph(
                              id="rtimpactdata",
                              figure = fig43
                          ),
                          dcc.Graph(
                              id="rtdamagedata",
                              figure= avg_linegraphs_real_time() 
                          )
                ]))
  
         ]),
    html.Div(id='outer-tab')
  ])

@app.callback(
    [Output('dd-output-container', 'children'), Output("graph_selected", "figure")],
    [Input('my_dropdown', 'value'), Input("time_interval", "value"), Input("tabs-example", "value")]
    
    )
     
def update_output(value, time_int, tabss):
    if value == "Sys2_SimFDD_xe":
      fig = graph3(time_int[0], time_int[1])
    elif value == "Sys2_Impact_Location_P2":
      fig = FDD_graphs()
    elif value == "Sys2_Total_Dmg":
      fig = avg_linegraphs(time_int[0], time_int[1])
    if tabss == "tab-1":
      tab_tr = html.Div([html.H3('Tab content 1')])
    elif tabss == "tab-2":
      tab_tr = html.Div([html.H3('Tab content 2')])
    

    return tab_tr, fig



app.run_server(debug=False, port=8040)


127.0.0.1 - - [21/Jul/2021 17:34:01] "GET /_shutdown_582c4451-2171-47a2-beb5-5d5c7dcf6eb8 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8040/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2021 17:34:02] "GET /_alive_582c4451-2171-47a2-beb5-5d5c7dcf6eb8 HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>